In [1]:
import polars as pl
from op_datasets.processing.execute import reader, BatchInput
from op_datasets.logic.transforms.daily_address_summary import (
    daily_address_summary,
    CONDITIONS,
)
from op_coreutils.logger import structlog
from op_coreutils.bigquery.write import overwrite_partitions

from datetime import datetime
import os

os.environ["OPLABS_ENV"] = prod


log = structlog.get_logger()

2024-10-16 19:00:24 [info     ] Loaded env var: OP_ANALYTICS_VAULT


In [2]:
inputbatch: BatchInput = None
for _inputbatch in reader(
    chain="op", block_spec="105376883:+500", source_spec="goldsky"
):
    inputbatch = _inputbatch
    break

dataframes = inputbatch.dataframes

2024-10-16 19:00:24 [info     ] Loaded 5 items into vault.    
2024-10-16 19:00:25 [info     ] Initialized Clickhouse client.
2024-10-16 19:00:26 [info     ] Query success: blocks in 1.81s 2000 rows
2024-10-16 19:00:27 [info     ] Query success: transactions in 2.46s 14166 rows
2024-10-16 19:00:27 [info     ] Query success: logs in 2.51s 59381 rows


In [3]:
dataframes

{'blocks': shape: (2_000, 24)
 ┌───────┬─────────┬──────────┬────────────┬───┬──────────┬───────────┬────────────┬────────────────┐
 │ chain ┆ network ┆ chain_id ┆ dt         ┆ … ┆ gas_used ┆ gas_limit ┆ extra_data ┆ transaction_co │
 │ ---   ┆ ---     ┆ ---      ┆ ---        ┆   ┆ ---      ┆ ---       ┆ ---        ┆ unt            │
 │ str   ┆ str     ┆ i32      ┆ str        ┆   ┆ i64      ┆ i64       ┆ str        ┆ ---            │
 │       ┆         ┆          ┆            ┆   ┆          ┆           ┆            ┆ i64            │
 ╞═══════╪═════════╪══════════╪════════════╪═══╪══════════╪═══════════╪════════════╪════════════════╡
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 676868   ┆ 30000000  ┆ 0x         ┆ 5              │
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 1149120  ┆ 30000000  ┆ 0x         ┆ 7              │
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 3368746  ┆ 30000000  ┆ 0x         ┆ 9              │
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 417

In [4]:
# check if our blocks are in the range we want

log.info(f"Min block number: {dataframes['blocks']['number'].min()}")
log.info(f"Max block number: {dataframes['blocks']['number'].max()}")
log.info(
    f"Min block number: {datetime.fromtimestamp(dataframes['blocks']['timestamp'].min())}"
)
log.info(
    f"Max block number: {datetime.fromtimestamp(dataframes['blocks']['timestamp'].max())}"
)

2024-10-16 19:00:27 [info     ] Min block number: 105376000   
2024-10-16 19:00:27 [info     ] Max block number: 105377999   
2024-10-16 19:00:27 [info     ] Min block number: 2023-06-09 18:46:17
2024-10-16 19:00:27 [info     ] Max block number: 2023-06-09 19:52:55


In [5]:
result = daily_address_summary(
    dataframes["transactions"],
    conditions=CONDITIONS,
    address="from_address",
    chain_id="chain_id",
    chain="chain",
    dt="dt",
)

In [6]:
# list(result.head()["hash"])
result.head()

from_address,chain_id,chain,dt,total_txs,total_txs_success,total_txs_fail,total_blocks,total_blocks_success,total_blocks_fail,min_block_number,max_block_number,block_interval_active,min_nonce,max_nonce,nonce_interval_active,total_l2_gas_used,total_l2_gas_used_success,total_l2_gas_used_fail,total_l1_gas_used,total_l1_gas_used_success,total_l1_gas_used_fail,total_gas_fees,total_gas_fees_success,total_gas_fees_fail,l2_contrib_gas_fees,l1_contrib_gas_fees,l1_blobgas_contrib_gas_fees,l1_l1gas_contrib_gas_fees,l2_contrib_gas_fees_base_fee,l2_contrib_gas_fees_priority_fee,avg_l2_gas_price_gwei,avg_l2_base_fee_gwei,avg_l2_priority_fee_gwei,avg_l1_gas_price_gwei,avg_l1_blob_base_fee_gwei,min_block_timestamp,max_block_timestamp,time_interval_active,unique_hours_active,perc_daily_hours_active,num_to_addresses,num_to_addresses_success,num_to_addresses_fail
str,i32,str,str,u32,u32,u32,u32,u32,u32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,"decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,17]","decimal[*,17]","decimal[*,17]","decimal[*,17]","decimal[*,17]",datetime[μs],datetime[μs],duration[μs],u32,f64,u32,u32,u32
"""0xdcd0ccf5ac9c9f2f65f4da06c8ca…",10,"""op""","""2023-06-09""",1,1,0,1,1,0,105377564,105377564,1,0,0,1,21000,21000,0,1900,1900,0,0.0000238703515274720000,0.0000238703515274720000,0.0000000000000000000000,0.0000000000127680000000,0.0000238703387594720000,0.0000000000000000000000,0.0000238703387594724045,0.0000000000127680000000,0.0000000000000000000000,0.00000060800000000,0.00000060800000000,0.00000000000000000,18.36745056900000000,0.00000000000000000,2023-06-09 23:38:25,2023-06-09 23:38:25,0µs,1,0.041667,1,1,0
"""0x0654f1957e5ab4837dc351ace517…",10,"""op""","""2023-06-09""",3,3,0,3,3,0,105376321,105376550,230,0,2,3,812294,812294,0,15788,15788,0,0.0001982178932599750000,0.0001982178932599750000,0.0000000000000000000000,0.0000000001210792180000,0.0001982177721807570000,0.0000000000000000000000,0.0001982177721807583753,0.0000000000466853610000,0.0000000000743938570000,0.00000014905836802,0.00000005747347758,0.00000009158489044,18.35521057713149227,0.00000000000000000,2023-06-09 22:56:59,2023-06-09 23:04:37,7m 38s,2,0.083333,3,3,0
"""0xd631465b98dd0df6bc368a9fd409…",10,"""op""","""2023-06-09""",1,1,0,1,1,0,105377119,105377119,1,1,1,1,221553,221553,0,4804,4804,0,0.0000595396660303510000,0.0000595396660303510000,0.0000000000000000000000,0.0000000000708969600000,0.0000595395951333910000,0.0000000000000000000000,0.0000595395951333910660,0.0000000000708969600000,0.0000000000000000000000,0.00000032000000000,0.00000032000000000,0.00000000000000000,18.11952367100000000,0.00000000000000000,2023-06-09 23:23:35,2023-06-09 23:23:35,0µs,1,0.041667,1,1,0
"""0xac73fa5bfdf8dafa2a56018653b8…",10,"""op""","""2023-06-09""",3,3,0,3,3,0,105376018,105376874,857,3,5,3,1390274,1390274,0,24456,24456,0,0.0004452100210582060000,0.0004452100210582060000,0.0000000000000000000000,0.0000000002342876020000,0.0004452097867706040000,0.0000000000000000000000,0.0004452097867706053624,0.0000000000749719020000,0.0000000001593157000000,0.00000016851901279,0.00000005392599012,0.00000011459302267,26.61479805064671246,0.00000000000000000,2023-06-09 22:46:53,2023-06-09 23:15:25,28m 32s,2,0.083333,2,2,0
"""0x7b4dddd2a05372c361ca5ad7d2b4…",10,"""op""","""2023-06-09""",1,1,0,1,1,0,105376894,105376894,1,0,0,1,36702,36702,0,2432,2432,0,0.0002308465629605320000,0.0002308465629605320000,0.0000000000000000000000,0.0001835100000000000000,0.0000473365629605320000,0.0000000000000000000000,0.0000473365629605329893,0.0001835100000000000000,0.0000000000000000000000,5.00000000000000000,5.00000000000000000,0.00000000000000000,28.45620945900000000,0.00000000000000000,2023-06-09 23:16:05,2023-06-09 23:16:05,0µs,1,0.041667,1,1,0


In [7]:
overwrite_partitions(result, "temp", "daily_address_summary", expiration_days=999)

2024-10-16 19:00:27 [info     ] Writing 1 partitions to BQ [2023-06-09 00:00:00 ... 2023-06-09 00:00:00]
2024-10-16 19:00:27 [info     ] DRYRUN OVERWRITE PARTITION: Wrote 2.9Krows 390.1KB to BQ temp.daily_address_summary
